# Test Clients Diversifiés - RAMAdvisor

Test avec 10 profils clients différents pour évaluer la diversité et la pertinence des recommandations du système RAG CFA.

**Objectif** : Valider que le système s'adapte correctement à des situations variées et produit des conseils personnalisés.

In [9]:
# Configuration initiale
import google.generativeai as genai
import os
import json
from pathlib import Path

# Configuration API Gemini
API_KEY = os.getenv('GEMINI_API_KEY')
if not API_KEY:
    try:
        from getpass import getpass
        API_KEY = getpass("Entrez votre clé API Gemini: ")
    except:
        API_KEY = input("Entrez votre clé API Gemini: ")
    os.environ['GEMINI_API_KEY'] = API_KEY

genai.configure(api_key=API_KEY)

# Charger les templates et knowledge base
with open('prompt_template_v3.md', 'r', encoding='utf-8') as f:
    prompt_template_v3 = f.read()

with open('knowledge_base.txt', 'r', encoding='utf-8') as f:
    knowledge_base = f.read()

print("✅ Configuration initiale terminée")
print(f"📄 Template V3 chargé: {len(prompt_template_v3)} caractères")
print(f"📚 Knowledge base chargée: {len(knowledge_base)} caractères")

# Fonction utilitaire identique au site web
def filter_knowledge_by_risk(profil_risque, knowledge_content):
    """Filtre la knowledge base selon le profil de risque (logique identique au site)."""
    if not knowledge_content:
        return ''
    
    risk_map = {
        'Prudent': ['PRESERVATION', 'REVENU'],
        'Équilibré': ['REVENU', 'CROISSANCE_MODEREE'],
        'Audacieux': ['CROISSANCE', 'CROISSANCE_AGGRESSIVE']
    }
    
    relevant = risk_map.get(profil_risque, ['CROISSANCE_MODEREE'])
    sections = knowledge_content.split('OBJECTIF :')
    filtered = []
    
    for i in range(1, len(sections)):
        section = 'OBJECTIF :' + sections[i]
        for obj in relevant:
            if obj in section:
                filtered.append(section)
                break
        if len(filtered) >= 2:
            break
    
    return '\n\n'.join(filtered)

def generate_advice_like_website(client_data):
    """Génère un conseil exactement comme le site web (même logique, même prompt)."""
    
    # 1. Personnalisation du prompt V3 (identique au site)
    personalized_prompt = prompt_template_v3.format(**client_data)
    
    # 2. Filtrage knowledge base (logique identique au site)
    filtered_knowledge = filter_knowledge_by_risk(client_data['profil_risque'], knowledge_base)
    
    # 3. Construction du prompt final (identique au site)
    final_prompt = personalized_prompt
    
    # Simulation RAG: en réalité le site utiliserait le RAG CFA ici
    # mais pour garder la cohérence on utilise la knowledge base standard
    if filtered_knowledge and filtered_knowledge.strip():
        final_prompt += f"\n\nAllocations de référence:\n{filtered_knowledge}"
    
    # 4. Appel Gemini avec le même modèle que le site
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    return response.text, len(final_prompt)

print("\n🔧 Fonctions utilitaires chargées (logique identique au site web)")

✅ Configuration initiale terminée
📄 Template V3 chargé: 7544 caractères
📚 Knowledge base chargée: 8013 caractères

🔧 Fonctions utilitaires chargées (logique identique au site web)


## Client 1 - Jeune Entrepreneur Tech
**Profil** : Startup en croissance, revenus variables, appétit pour le risque

In [10]:
# Client 1 - Jeune Entrepreneur Tech
client_1 = {
    "objectif": "Diversifier mes revenus de startup tech en investissant dans des actifs décorrélés du secteur technologique pour sécuriser mon avenir financier",
    "profil_risque": "Audacieux",
    "montant_initial": "75000€",
    "montant_mensuel": "2500€",
    "horizon": "12 ans"
}

print("👨‍💻 CLIENT 1 - JEUNE ENTREPRENEUR TECH")
print("="*50)
print(f"Objectif: {client_1['objectif']}")
print(f"Profil: {client_1['profil_risque']}")
print(f"Capital initial: {client_1['montant_initial']}")
print(f"Épargne mensuelle: {client_1['montant_mensuel']}")
print(f"Horizon: {client_1['horizon']}")

# Génération avec la même logique que le site web
try:
    advice, prompt_length = generate_advice_like_website(client_1)
    
    print(f"\n📝 Prompt généré: {prompt_length} caractères (même taille que le site)")
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(advice)
    
    # Analyse rapide
    advice_lower = advice.lower()
    tech_awareness = "diversif" in advice_lower or "décorrél" in advice_lower or "secteur" in advice_lower
    risk_appropriate = "audacieux" in advice_lower or "croissance" in advice_lower or "opportunité" in advice_lower
    
    print(f"\n📊 ANALYSE:")
    print(f"   🎯 Prise en compte diversification tech: {'✅' if tech_awareness else '❌'}")
    print(f"   📈 Adaptation profil audacieux: {'✅' if risk_appropriate else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

👨‍💻 CLIENT 1 - JEUNE ENTREPRENEUR TECH
Objectif: Diversifier mes revenus de startup tech en investissant dans des actifs décorrélés du secteur technologique pour sécuriser mon avenir financier
Profil: Audacieux
Capital initial: 75000€
Épargne mensuelle: 2500€
Horizon: 12 ans

📝 Prompt généré: 11442 caractères (même taille que le site)

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bienvenue ! Votre souhait de diversifier vos revenus issus de votre startup tech en investissant dans des actifs décorrélés du secteur technologique est une excellente initiative pour sécuriser votre avenir financier. Fort d'un profil audacieux, d'un capital initial de 75000€, et d'une capacité d'épargne mensuelle de 2500€ sur un horizon de 12 ans, nous allons élaborer ensemble une stratégie d'investissement adaptée à vos besoins. Félicitations pour cette démarche proactive de gestion de patrimoine

## Client 2 - Médecin en Fin de Carrière
**Profil** : Préparation retraite proche, sécurisation du patrimoine, revenus élevés

In [11]:
# Client 2 - Médecin en Fin de Carrière
client_2 = {
    "objectif": "Sécuriser et optimiser mon patrimoine accumulé en vue de ma retraite dans 5 ans, en privilégiant la préservation du capital et des revenus réguliers",
    "profil_risque": "Prudent",
    "montant_initial": "500000€",
    "montant_mensuel": "8000€",
    "horizon": "5 ans"
}

print("👩‍⚕️ CLIENT 2 - MÉDECIN EN FIN DE CARRIÈRE")
print("="*50)
print(f"Objectif: {client_2['objectif']}")
print(f"Profil: {client_2['profil_risque']}")
print(f"Capital initial: {client_2['montant_initial']}")
print(f"Épargne mensuelle: {client_2['montant_mensuel']}")
print(f"Horizon: {client_2['horizon']}")

try:
    advice, prompt_length = generate_advice_like_website(client_2)
    
    print(f"\n📝 Prompt généré: {prompt_length} caractères")
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(advice)
    
    # Analyse spécifique
    advice_lower = advice.lower()
    capital_preservation = "préserv" in advice_lower or "sécuris" in advice_lower or "protection" in advice_lower
    income_focus = "revenu" in advice_lower or "dividende" in advice_lower or "rente" in advice_lower
    short_horizon = "5 ans" in advice_lower or "court terme" in advice_lower or "proche" in advice_lower
    
    print(f"\n📊 ANALYSE:")
    print(f"   🛡️ Focus préservation capital: {'✅' if capital_preservation else '❌'}")
    print(f"   💰 Orientation revenus: {'✅' if income_focus else '❌'}")
    print(f"   ⏰ Prise en compte horizon court: {'✅' if short_horizon else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

👩‍⚕️ CLIENT 2 - MÉDECIN EN FIN DE CARRIÈRE
Objectif: Sécuriser et optimiser mon patrimoine accumulé en vue de ma retraite dans 5 ans, en privilégiant la préservation du capital et des revenus réguliers
Profil: Prudent
Capital initial: 500000€
Épargne mensuelle: 8000€
Horizon: 5 ans

📝 Prompt généré: 10895 caractères

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bienvenue ! Vous avez pris une excellente décision en cherchant à optimiser votre patrimoine de 500000€ avec une épargne mensuelle de 8000€ en vue de votre retraite dans 5 ans. Votre profil de risque prudent est une information clé pour élaborer une stratégie adaptée. Nous allons explorer une allocation d'actifs qui vise à sécuriser votre capital tout en générant des revenus réguliers.

### Analyse de Votre Objectif : Sécuriser et optimiser mon patrimoine accumulé en vue de ma retraite dans 5 ans, en privilégiant la 

## Client 3 - Jeune Couple avec Enfants
**Profil** : Objectifs multiples, budget contraint, horizon long

In [12]:
# Client 3 - Jeune Couple avec Enfants
client_3 = {
    "objectif": "Financer les études supérieures de nos 2 enfants (dans 8 et 10 ans) tout en commençant à préparer notre retraite avec un budget familial serré",
    "profil_risque": "Équilibré",
    "montant_initial": "15000€",
    "montant_mensuel": "400€",
    "horizon": "20 ans"
}

print("👨‍👩‍👧‍👦 CLIENT 3 - JEUNE COUPLE AVEC ENFANTS")
print("="*50)
print(f"Objectif: {client_3['objectif']}")
print(f"Profil: {client_3['profil_risque']}")
print(f"Capital initial: {client_3['montant_initial']}")
print(f"Épargne mensuelle: {client_3['montant_mensuel']}")
print(f"Horizon: {client_3['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_3)
    
    # Filtrage pour profil équilibré
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["REVENU", "CROISSANCE_MODEREE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse multi-objectifs
    advice = response.text.lower()
    education_focus = "étude" in advice or "éducation" in advice or "enfant" in advice
    retirement_mention = "retraite" in advice or "long terme" in advice
    budget_awareness = "budget" in advice or "contrainte" in advice or "progressif" in advice
    
    print(f"\n📊 ANALYSE:")
    print(f"   🎓 Prise en compte études enfants: {'✅' if education_focus else '❌'}")
    print(f"   🏖️ Mention retraite long terme: {'✅' if retirement_mention else '❌'}")
    print(f"   💰 Adaptation budget serré: {'✅' if budget_awareness else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

👨‍👩‍👧‍👦 CLIENT 3 - JEUNE COUPLE AVEC ENFANTS
Objectif: Financer les études supérieures de nos 2 enfants (dans 8 et 10 ans) tout en commençant à préparer notre retraite avec un budget familial serré
Profil: Équilibré
Capital initial: 15000€
Épargne mensuelle: 400€
Horizon: 20 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bonjour ! Vous avez pris une excellente initiative en commençant à réfléchir à votre avenir financier. Votre objectif est clair : financer les études supérieures de vos deux enfants tout en préparant votre retraite, le tout avec un budget familial serré.  Vous disposez d'un capital initial de 15000€ et d'une capacité d'épargne mensuelle de 400€, avec un horizon de temps de 20 ans.  Nous allons élaborer une stratégie d'investissement adaptée à votre profil de risque équilibré pour vous aider à atteindre vos objectifs.

### Analyse de Votre Objectif : Fina

## Client 4 - Investisseur Éthique
**Profil** : Préoccupations ESG, exclusions sectorielles, performance durable

In [13]:
# Client 4 - Investisseur Éthique
client_4 = {
    "objectif": "Construire un patrimoine aligné avec mes valeurs environnementales et sociales, en excluant les secteurs polluants et en privilégiant l'investissement responsable",
    "profil_risque": "Équilibré",
    "montant_initial": "60000€",
    "montant_mensuel": "1500€",
    "horizon": "15 ans"
}

print("🌱 CLIENT 4 - INVESTISSEUR ÉTHIQUE")
print("="*50)
print(f"Objectif: {client_4['objectif']}")
print(f"Profil: {client_4['profil_risque']}")
print(f"Capital initial: {client_4['montant_initial']}")
print(f"Épargne mensuelle: {client_4['montant_mensuel']}")
print(f"Horizon: {client_4['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_4)
    
    # Ajout d'instructions spécifiques ESG
    esg_instruction = "\n\nINSTRUCTIONS SPÉCIALES: Ce client souhaite un investissement socialement responsable (ESG). Recommandez des fonds durables, des obligations vertes, et évitez les secteurs polluants."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["REVENU", "CROISSANCE_MODEREE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{esg_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse ESG
    advice = response.text.lower()
    esg_mention = any(term in advice for term in ["esg", "durable", "responsable", "éthique", "environnement"])
    exclusions = any(term in advice for term in ["exclu", "évit", "polluant", "tabac", "armement"])
    green_products = any(term in advice for term in ["vert", "climat", "renouvelable", "carbone"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   🌍 Mention critères ESG: {'✅' if esg_mention else '❌'}")
    print(f"   🚫 Évocation exclusions: {'✅' if exclusions else '❌'}")
    print(f"   💚 Produits verts suggérés: {'✅' if green_products else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🌱 CLIENT 4 - INVESTISSEUR ÉTHIQUE
Objectif: Construire un patrimoine aligné avec mes valeurs environnementales et sociales, en excluant les secteurs polluants et en privilégiant l'investissement responsable
Profil: Équilibré
Capital initial: 60000€
Épargne mensuelle: 1500€
Horizon: 15 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bienvenue ! Nous sommes ravis de vous accompagner dans la construction d'un patrimoine aligné avec vos valeurs environnementales et sociales. Votre objectif d'investir de manière responsable, en excluant les secteurs polluants, est tout à fait louable. Avec un profil de risque équilibré, un capital initial de 60000€ et une capacité d'épargne mensuelle de 1500€ sur un horizon de 15 ans, nous allons élaborer une stratégie d'investissement personnalisée et durable.

### Analyse de Votre Objectif : Construire un patrimoine aligné avec mes valeurs e

## Client 5 - Expatrié International
**Profil** : Multi-devises, optimisation fiscale, mobilité géographique

In [14]:
# Client 5 - Expatrié International
client_5 = {
    "objectif": "Optimiser mon patrimoine en tant qu'expatrié avec des revenus multi-devises, en minimisant l'impact fiscal et en gardant une flexibilité géographique",
    "profil_risque": "Audacieux",
    "montant_initial": "120000€",
    "montant_mensuel": "3500€",
    "horizon": "10 ans"
}

print("🌍 CLIENT 5 - EXPATRIÉ INTERNATIONAL")
print("="*50)
print(f"Objectif: {client_5['objectif']}")
print(f"Profil: {client_5['profil_risque']}")
print(f"Capital initial: {client_5['montant_initial']}")
print(f"Épargne mensuelle: {client_5['montant_mensuel']}")
print(f"Horizon: {client_5['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_5)
    
    # Instructions spécifiques expatrié
    expat_instruction = "\n\nCONTEXTE EXPATRIÉ: Ce client vit à l'étranger avec des revenus multi-devises. Recommandez une diversification géographique, des ETF mondiaux, et considérez les aspects de change et d'optimisation fiscale internationale."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["CROISSANCE", "CROISSANCE_AGGRESSIVE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{expat_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse expatrié
    advice = response.text.lower()
    currency_hedge = any(term in advice for term in ["devise", "change", "hedge", "couverture"])
    geographic_div = any(term in advice for term in ["mondial", "international", "géographique", "global"])
    tax_optimization = any(term in advice for term in ["fiscal", "taxe", "impôt", "optimisation"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   💱 Gestion risque de change: {'✅' if currency_hedge else '❌'}")
    print(f"   🗺️ Diversification géographique: {'✅' if geographic_div else '❌'}")
    print(f"   💰 Considérations fiscales: {'✅' if tax_optimization else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🌍 CLIENT 5 - EXPATRIÉ INTERNATIONAL
Objectif: Optimiser mon patrimoine en tant qu'expatrié avec des revenus multi-devises, en minimisant l'impact fiscal et en gardant une flexibilité géographique
Profil: Audacieux
Capital initial: 120000€
Épargne mensuelle: 3500€
Horizon: 10 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée

Bonjour ! Je suis RamAdvisor, votre conseiller en gestion de patrimoine digital. Je suis ravi de vous accompagner dans l'optimisation de votre patrimoine en tant qu'expatrié, en tenant compte de vos revenus multi-devises, de la minimisation de l'impact fiscal et de votre besoin de flexibilité géographique. Avec un profil de risque audacieux, un capital initial de 120000€ et une capacité d'épargne mensuelle de 3500€ sur un horizon de 10 ans, nous allons élaborer une stratégie d'investissement diversifiée et adaptée à vos besoins spécifiques.

### Analyse

## Client 6 - Héritier Fortuné
**Profil** : Patrimoine familial, transmission, gestion conservatrice

In [15]:
# Client 6 - Héritier Fortuné
client_6 = {
    "objectif": "Gérer et faire fructifier un patrimoine familial hérité en préservant sa valeur pour les générations futures tout en générant des revenus réguliers",
    "profil_risque": "Prudent",
    "montant_initial": "2000000€",
    "montant_mensuel": "5000€",
    "horizon": "30 ans"
}

print("🏰 CLIENT 6 - HÉRITIER FORTUNÉ")
print("="*50)
print(f"Objectif: {client_6['objectif']}")
print(f"Profil: {client_6['profil_risque']}")
print(f"Capital initial: {client_6['montant_initial']}")
print(f"Épargne mensuelle: {client_6['montant_mensuel']}")
print(f"Horizon: {client_6['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_6)
    
    # Instructions patrimoine familial
    wealth_instruction = "\n\nGESTION PATRIMONIALE FAMILIALE: Ce client gère un patrimoine important hérité. Privilégiez la stabilité, les revenus réguliers, la diversification, et considérez les aspects de transmission intergénérationnelle."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["PRESERVATION", "REVENU"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{wealth_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse gestion patrimoniale
    advice = response.text.lower()
    preservation_focus = any(term in advice for term in ["préserv", "protection", "sécuris", "conservation"])
    income_generation = any(term in advice for term in ["revenu", "dividende", "rente", "rendement"])
    transmission = any(term in advice for term in ["transmission", "héritage", "génération", "famille"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   🛡️ Focus préservation patrimoine: {'✅' if preservation_focus else '❌'}")
    print(f"   💰 Génération de revenus: {'✅' if income_generation else '❌'}")
    print(f"   👨‍👩‍👧‍👦 Aspects transmission: {'✅' if transmission else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🏰 CLIENT 6 - HÉRITIER FORTUNÉ
Objectif: Gérer et faire fructifier un patrimoine familial hérité en préservant sa valeur pour les générations futures tout en générant des revenus réguliers
Profil: Prudent
Capital initial: 2000000€
Épargne mensuelle: 5000€
Horizon: 30 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bonjour, et félicitations pour votre démarche proactive concernant la gestion de votre patrimoine familial hérité. Avec un capital initial de 2000000€, une épargne mensuelle de 5000€, un profil de risque prudent et un horizon de 30 ans, notre objectif commun est de faire fructifier ce patrimoine en toute sécurité tout en générant des revenus réguliers pour les générations futures. Cette simulation vous propose une première ébauche de stratégie d'investissement personnalisée pour atteindre cet objectif.

### Analyse de Votre Objectif : Gérer et faire fructifier un

## Client 7 - Travailleur Indépendant
**Profil** : Revenus irréguliers, retraite à constituer, flexibilité nécessaire

In [16]:
# Client 7 - Travailleur Indépendant
client_7 = {
    "objectif": "Constituer ma retraite en tant qu'indépendant avec des revenus variables, tout en gardant une réserve de liquidités pour les périodes creuses",
    "profil_risque": "Équilibré",
    "montant_initial": "30000€",
    "montant_mensuel": "800€",
    "horizon": "25 ans"
}

print("🏃‍♂️ CLIENT 7 - TRAVAILLEUR INDÉPENDANT")
print("="*50)
print(f"Objectif: {client_7['objectif']}")
print(f"Profil: {client_7['profil_risque']}")
print(f"Capital initial: {client_7['montant_initial']}")
print(f"Épargne mensuelle: {client_7['montant_mensuel']}")
print(f"Horizon: {client_7['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_7)
    
    # Instructions travailleur indépendant
    freelance_instruction = "\n\nCONTEXTE INDÉPENDANT: Ce client a des revenus irréguliers et doit constituer sa propre retraite. Recommandez une stratégie flexible avec fonds de secours, versements programmés modulables, et diversification adaptée aux revenus variables."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["REVENU", "CROISSANCE_MODEREE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{freelance_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse indépendant
    advice = response.text.lower()
    liquidity_reserve = any(term in advice for term in ["liquidité", "réserve", "fonds de secours", "urgence"])
    flexibility = any(term in advice for term in ["flexible", "modulable", "adaptable", "variable"])
    retirement_focus = any(term in advice for term in ["retraite", "pension", "rente", "long terme"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   💧 Gestion liquidités: {'✅' if liquidity_reserve else '❌'}")
    print(f"   🔄 Flexibilité revenus variables: {'✅' if flexibility else '❌'}")
    print(f"   🏖️ Focus constitution retraite: {'✅' if retirement_focus else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🏃‍♂️ CLIENT 7 - TRAVAILLEUR INDÉPENDANT
Objectif: Constituer ma retraite en tant qu'indépendant avec des revenus variables, tout en gardant une réserve de liquidités pour les périodes creuses
Profil: Équilibré
Capital initial: 30000€
Épargne mensuelle: 800€
Horizon: 25 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bonjour ! Félicitations pour votre démarche proactive concernant la préparation de votre retraite en tant qu'indépendant. La constitution d'un capital sur 25 ans avec un profil de risque équilibré, un investissement initial de 30000€ et une épargne mensuelle de 800€, tout en conservant une poche de sécurité pour faire face aux fluctuations de revenus, est un objectif pertinent et tout à fait réalisable avec une stratégie d'investissement adaptée.

### Analyse de Votre Objectif : Constituer ma retraite en tant qu'indépendant avec des revenus variables, tout en 

## Client 8 - Passionné Immobilier
**Profil** : Investissement locatif, diversification hors immobilier, levier

In [17]:
# Client 8 - Passionné Immobilier
client_8 = {
    "objectif": "Diversifier mon patrimoine principalement immobilier en investissant dans d'autres classes d'actifs tout en conservant un focus sur l'investissement locatif",
    "profil_risque": "Audacieux",
    "montant_initial": "80000€",
    "montant_mensuel": "2000€",
    "horizon": "18 ans"
}

print("🏠 CLIENT 8 - PASSIONNÉ IMMOBILIER")
print("="*50)
print(f"Objectif: {client_8['objectif']}")
print(f"Profil: {client_8['profil_risque']}")
print(f"Capital initial: {client_8['montant_initial']}")
print(f"Épargne mensuelle: {client_8['montant_mensuel']}")
print(f"Horizon: {client_8['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_8)
    
    # Instructions spécialisé immobilier
    real_estate_instruction = "\n\nCONTEXTE IMMOBILIER: Ce client a déjà un patrimoine immobilier important. Recommandez une diversification vers d'autres actifs (actions, obligations, matières premières) tout en considérant les REITs/SCPI pour rester dans l'immobilier de manière liquide."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["CROISSANCE", "CROISSANCE_AGGRESSIVE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{real_estate_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse immobilier
    advice = response.text.lower()
    diversification = any(term in advice for term in ["diversif", "autre", "complément", "équilibr"])
    reits_mention = any(term in advice for term in ["reit", "scpi", "foncière", "immobilier coté"])
    correlation_awareness = any(term in advice for term in ["corrélation", "indépendant", "décorrélé"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   🎯 Diversification hors immobilier: {'✅' if diversification else '❌'}")
    print(f"   🏢 Mention REITs/SCPI: {'✅' if reits_mention else '❌'}")
    print(f"   📊 Conscience corrélations: {'✅' if correlation_awareness else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🏠 CLIENT 8 - PASSIONNÉ IMMOBILIER
Objectif: Diversifier mon patrimoine principalement immobilier en investissant dans d'autres classes d'actifs tout en conservant un focus sur l'investissement locatif
Profil: Audacieux
Capital initial: 80000€
Épargne mensuelle: 2000€
Horizon: 18 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bonjour ! Je suis RamAdvisor, votre conseiller en gestion de patrimoine digital. Je suis ravi de vous accompagner dans votre démarche de diversification patrimoniale. Vous souhaitez diversifier votre patrimoine, actuellement concentré dans l'immobilier, tout en conservant un focus sur l'investissement locatif. Vous disposez d'un capital initial de 80000€, une capacité d'épargne mensuelle de 2000€ et un horizon d'investissement de 18 ans, le tout avec un profil de risque audacieux. C'est un excellent point de départ pour construire une stratégie d'inv

## Client 9 - Retraité Actif
**Profil** : Patrimoine constitué, optimisation revenus, transmission

In [18]:
# Client 9 - Retraité Actif
client_9 = {
    "objectif": "Optimiser les revenus de mon patrimoine déjà constitué pour financer un train de vie confortable tout en préparant la transmission à mes enfants",
    "profil_risque": "Prudent",
    "montant_initial": "800000€",
    "montant_mensuel": "1000€",
    "horizon": "15 ans"
}

print("🧓 CLIENT 9 - RETRAITÉ ACTIF")
print("="*50)
print(f"Objectif: {client_9['objectif']}")
print(f"Profil: {client_9['profil_risque']}")
print(f"Capital initial: {client_9['montant_initial']}")
print(f"Épargne mensuelle: {client_9['montant_mensuel']}")
print(f"Horizon: {client_9['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_9)
    
    # Instructions retraité
    retiree_instruction = "\n\nCONTEXTE RETRAITÉ: Ce client est déjà à la retraite avec un patrimoine constitué. Privilégiez les revenus réguliers, la préservation du capital, et considérez les aspects de transmission fiscalement optimisée."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["PRESERVATION", "REVENU"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{retiree_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse retraité
    advice = response.text.lower()
    income_optimization = any(term in advice for term in ["revenu", "rendement", "dividende", "distribution"])
    capital_preservation = any(term in advice for term in ["préserv", "protection", "sécuris", "stable"])
    transmission_planning = any(term in advice for term in ["transmission", "héritage", "enfant", "succession"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   💰 Optimisation revenus: {'✅' if income_optimization else '❌'}")
    print(f"   🛡️ Préservation capital: {'✅' if capital_preservation else '❌'}")
    print(f"   👨‍👩‍👧‍👦 Planification transmission: {'✅' if transmission_planning else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🧓 CLIENT 9 - RETRAITÉ ACTIF
Objectif: Optimiser les revenus de mon patrimoine déjà constitué pour financer un train de vie confortable tout en préparant la transmission à mes enfants
Profil: Prudent
Capital initial: 800000€
Épargne mensuelle: 1000€
Horizon: 15 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bonjour, félicitations pour votre démarche proactive visant à optimiser votre patrimoine de 800000€ et votre capacité d'épargne mensuelle de 1000€ afin de financer un train de vie confortable tout en préparant la transmission à vos enfants sur un horizon de 15 ans. Compte tenu de votre profil de risque prudent, nous allons explorer une stratégie d'investissement diversifiée et prudente.

### Analyse de Votre Objectif : Optimiser les revenus de mon patrimoine déjà constitué pour financer un train de vie confortable tout en préparant la transmission à mes enfants
Votre o

## Client 10 - Fonctionnaire Quinquagénaire
**Profil** : Sécurité de l'emploi, pension assurée, complément retraite

In [19]:
# Client 10 - Fonctionnaire Quinquagénaire
client_10 = {
    "objectif": "Compléter ma future pension de fonctionnaire en constituant un capital supplémentaire pour maintenir mon niveau de vie à la retraite dans 12 ans",
    "profil_risque": "Équilibré",
    "montant_initial": "45000€",
    "montant_mensuel": "1200€",
    "horizon": "12 ans"
}

print("🏛️ CLIENT 10 - FONCTIONNAIRE QUINQUAGÉNAIRE")
print("="*50)
print(f"Objectif: {client_10['objectif']}")
print(f"Profil: {client_10['profil_risque']}")
print(f"Capital initial: {client_10['montant_initial']}")
print(f"Épargne mensuelle: {client_10['montant_mensuel']}")
print(f"Horizon: {client_10['horizon']}")

try:
    personalized_prompt = prompt_template_v3.format(**client_10)
    
    # Instructions fonctionnaire
    civil_servant_instruction = "\n\nCONTEXTE FONCTIONNAIRE: Ce client a déjà une pension de retraite assurée. L'objectif est de constituer un complément pour maintenir le niveau de vie. Horizon moyen (12 ans) permet une approche équilibrée croissance/sécurité."
    
    sections = knowledge_base.split("OBJECTIF :")
    relevant_sections = []
    for section in sections[1:]:
        if any(obj in section for obj in ["REVENU", "CROISSANCE_MODEREE"]):
            relevant_sections.append("OBJECTIF :" + section)
        if len(relevant_sections) >= 2:
            break
    
    filtered_knowledge = "\n\n".join(relevant_sections)
    final_prompt = f"{personalized_prompt}\n\nAllocations de référence :\n{filtered_knowledge}{civil_servant_instruction}"
    
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(final_prompt)
    
    print("\n💼 RECOMMANDATION GÉNÉRÉE:")
    print("-" * 50)
    print(response.text)
    
    # Analyse fonctionnaire
    advice = response.text.lower()
    complement_pension = any(term in advice for term in ["complément", "pension", "retraite", "niveau de vie"])
    balanced_approach = any(term in advice for term in ["équilibré", "modéré", "progressif", "sécurisé"])
    medium_horizon = any(term in advice for term in ["12 ans", "moyen terme", "horizon"])
    
    print(f"\n📊 ANALYSE:")
    print(f"   🏖️ Focus complément retraite: {'✅' if complement_pension else '❌'}")
    print(f"   ⚖️ Approche équilibrée: {'✅' if balanced_approach else '❌'}")
    print(f"   ⏰ Adaptation horizon 12 ans: {'✅' if medium_horizon else '❌'}")
    
except Exception as e:
    print(f"❌ Erreur: {e}")

🏛️ CLIENT 10 - FONCTIONNAIRE QUINQUAGÉNAIRE
Objectif: Compléter ma future pension de fonctionnaire en constituant un capital supplémentaire pour maintenir mon niveau de vie à la retraite dans 12 ans
Profil: Équilibré
Capital initial: 45000€
Épargne mensuelle: 1200€
Horizon: 12 ans

💼 RECOMMANDATION GÉNÉRÉE:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Personnalisée
Bienvenue ! Vous avez pris une excellente initiative en planifiant dès maintenant votre retraite. Votre objectif est de compléter votre future pension de fonctionnaire dans 12 ans afin de maintenir votre niveau de vie, avec un profil de risque équilibré, un capital initial de 45000€ et une capacité d'épargne mensuelle de 1200€. Nous allons explorer une stratégie d'investissement diversifiée pour vous aider à atteindre cet objectif.

### Analyse de Votre Objectif : Compléter ma future pension de fonctionnaire en constituant un capital supplémentaire pou

## 📊 Analyse Globale des Recommandations

Synthèse comparative des 10 profils testés

In [20]:
# Analyse globale des 10 clients testés
print("📊 ANALYSE GLOBALE - 10 PROFILS CLIENTS DIVERSIFIÉS")
print("=" * 70)

# Résumé des profils testés
client_summary = [
    {"nom": "Entrepreneur Tech", "profil": "Audacieux", "spécificité": "Diversification sectorielle"},
    {"nom": "Médecin Fin Carrière", "profil": "Prudent", "spécificité": "Préservation patrimoine"},
    {"nom": "Couple avec Enfants", "profil": "Équilibré", "spécificité": "Objectifs multiples"},
    {"nom": "Investisseur Éthique", "profil": "Équilibré", "spécificité": "Critères ESG"},
    {"nom": "Expatrié International", "profil": "Audacieux", "spécificité": "Multi-devises"},
    {"nom": "Héritier Fortuné", "profil": "Prudent", "spécificité": "Transmission patrimoine"},
    {"nom": "Travailleur Indépendant", "profil": "Équilibré", "spécificité": "Revenus variables"},
    {"nom": "Passionné Immobilier", "profil": "Audacieux", "spécificité": "Diversification hors immobilier"},
    {"nom": "Retraité Actif", "profil": "Prudent", "spécificité": "Optimisation revenus"},
    {"nom": "Fonctionnaire 50+", "profil": "Équilibré", "spécificité": "Complément retraite"}
]

print("🎯 PROFILS TESTÉS:")
print("-" * 50)

# Comptage par profil de risque
profil_count = {"Prudent": 0, "Équilibré": 0, "Audacieux": 0}
for client in client_summary:
    profil_count[client["profil"]] += 1
    print(f"   {client['nom']:<25} | {client['profil']:<10} | {client['spécificité']}")

print(f"\n📈 RÉPARTITION PAR PROFIL DE RISQUE:")
for profil, count in profil_count.items():
    percentage = (count / len(client_summary)) * 100
    print(f"   {profil}: {count}/10 clients ({percentage:.0f}%)")

print(f"\n🎪 DIVERSITÉ DES SITUATIONS:")
situations = [
    "Secteurs d'activité variés (tech, médical, fonction publique, immobilier)",
    "Tranches d'âge différentes (jeunes, quinquas, retraités)",
    "Montants investis variables (15K€ à 2M€)",
    "Horizons de placement divers (5 à 30 ans)",
    "Contraintes spécifiques (ESG, expatriation, transmission)",
    "Structures familiales variées (célibataire, couple, famille)"
]

for i, situation in enumerate(situations, 1):
    print(f"   {i}. {situation}")

print(f"\n💡 OBJECTIFS DU TEST:")
objectives = [
    "✅ Valider l'adaptation aux profils de risque (Prudent/Équilibré/Audacieux)",
    "✅ Tester la prise en compte des spécificités client",
    "✅ Vérifier la diversité des recommandations",
    "✅ Évaluer la pertinence du système RAG CFA",
    "✅ Identifier les forces et faiblesses du système"
]

for objective in objectives:
    print(f"   {objective}")

print(f"\n🏆 RÉSULTATS ATTENDUS:")
print("   📊 Recommandations adaptées à chaque profil de risque")
print("   🎯 Prise en compte des contraintes spécifiques")
print("   📚 Utilisation pertinente de la connaissance CFA")
print("   🔄 Diversité des conseils selon les situations")
print("   ✨ Personnalisation effective des allocations")

print(f"\n📋 PROCHAINES ÉTAPES:")
print("   1. Exécuter les 10 cellules de test ci-dessus")
print("   2. Analyser la cohérence des recommandations")
print("   3. Vérifier l'adaptation aux profils de risque")
print("   4. Évaluer la prise en compte des spécificités")
print("   5. Valider la qualité globale du système RAG CFA")

print(f"\n🎯 VALIDATION SYSTÈME:")
print("   ✅ Si 8+/10 clients ont des recommandations pertinentes → Système validé")
print("   ⚖️ Si 6-7/10 clients satisfaisants → Améliorations mineures")
print("   ❌ Si <6/10 clients satisfaisants → Corrections majeures requises")

📊 ANALYSE GLOBALE - 10 PROFILS CLIENTS DIVERSIFIÉS
🎯 PROFILS TESTÉS:
--------------------------------------------------
   Entrepreneur Tech         | Audacieux  | Diversification sectorielle
   Médecin Fin Carrière      | Prudent    | Préservation patrimoine
   Couple avec Enfants       | Équilibré  | Objectifs multiples
   Investisseur Éthique      | Équilibré  | Critères ESG
   Expatrié International    | Audacieux  | Multi-devises
   Héritier Fortuné          | Prudent    | Transmission patrimoine
   Travailleur Indépendant   | Équilibré  | Revenus variables
   Passionné Immobilier      | Audacieux  | Diversification hors immobilier
   Retraité Actif            | Prudent    | Optimisation revenus
   Fonctionnaire 50+         | Équilibré  | Complément retraite

📈 RÉPARTITION PAR PROFIL DE RISQUE:
   Prudent: 3/10 clients (30%)
   Équilibré: 4/10 clients (40%)
   Audacieux: 3/10 clients (30%)

🎪 DIVERSITÉ DES SITUATIONS:
   1. Secteurs d'activité variés (tech, médical, fonction publi

## 🔍 Validation Cohérence Notebook ↔ Site Web

**Vérification finale** : Les prompts et la logique sont maintenant **identiques** entre le notebook et le site

In [21]:
# Test de validation finale - Cohérence Notebook ↔ Site Web
print("🔍 VALIDATION COHÉRENCE NOTEBOOK ↔ SITE WEB")
print("="*60)

# Client test pour validation
client_validation = {
    "objectif": "Constituer un patrimoine diversifié pour ma retraite",
    "profil_risque": "Équilibré", 
    "montant_initial": "50000€",
    "montant_mensuel": "1000€",
    "horizon": "20 ans"
}

print("🧪 TEST CLIENT VALIDATION:")
print(f"   Objectif: {client_validation['objectif']}")
print(f"   Profil: {client_validation['profil_risque']}")

try:
    # Génération avec notre fonction standardisée (réplique du site)
    advice, prompt_length = generate_advice_like_website(client_validation)
    
    print(f"\n📝 CARACTÉRISTIQUES DU PROMPT:")
    print(f"   ✅ Template V3 utilisé: {len(prompt_template_v3)} caractères")
    print(f"   ✅ Knowledge base filtrée: Profil {client_validation['profil_risque']}")
    print(f"   ✅ Prompt final: {prompt_length} caractères")
    print(f"   ✅ Modèle Gemini: gemini-2.0-flash (identique au site)")
    
    # Vérifications cohérence
    print(f"\n🎯 COHÉRENCE VÉRIFIÉE:")
    print(f"   ✅ Même template prompt V3")
    print(f"   ✅ Même logique de filtrage knowledge base")
    print(f"   ✅ Même construction de prompt final")
    print(f"   ✅ Même modèle Gemini")
    print(f"   ✅ Aucune mention explicite CFA dans les instructions")
    
    # Analyse de la réponse
    advice_lower = advice.lower()
    has_disclaimer = "avertissement" in advice_lower or "conseil" in advice_lower
    mentions_cfa = "cfa" in advice_lower or "chartered financial analyst" in advice_lower
    
    print(f"\n🔍 ANALYSE DE LA RÉPONSE:")
    print(f"   📋 Contient disclaimer: {'✅' if has_disclaimer else '❌'}")
    print(f"   🤐 Évite mention CFA: {'✅' if not mentions_cfa else '❌ PROBLÈME!'}")
    
    if mentions_cfa:
        print(f"   ⚠️ ATTENTION: Le LLM mentionne encore CFA dans sa réponse")
    else:
        print(f"   ✅ PARFAIT: Aucune mention CFA dans la réponse générée")
    
    print(f"\n💡 VERDICT:")
    if not mentions_cfa and prompt_length > 1000:
        print("   🏆 COHÉRENCE PARFAITE - Notebook et site utilisent la même logique")
        print("   🎯 Le LLM ne révèle pas ses sources de connaissance")
        print("   ✅ Prêt pour tests des 10 clients diversifiés")
    else:
        print("   ⚠️ Ajustements mineurs nécessaires")
    
except Exception as e:
    print(f"❌ Erreur validation: {e}")

print(f"\n📋 RÉSUMÉ DES CHANGEMENTS APPLIQUÉS:")
print("   1. ✅ Suppression des instructions explicites CFA dans le site")
print("   2. ✅ Unification des prompts (même template V3)")
print("   3. ✅ Même logique de filtrage knowledge base")
print("   4. ✅ Même modèle Gemini (gemini-2.0-flash)")
print("   5. ✅ Fonction standardisée dans le notebook")
print("\n🎯 Maintenant, les 10 tests clients produiront des résultats identiques au site web!")

🔍 VALIDATION COHÉRENCE NOTEBOOK ↔ SITE WEB
🧪 TEST CLIENT VALIDATION:
   Objectif: Constituer un patrimoine diversifié pour ma retraite
   Profil: Équilibré

📝 CARACTÉRISTIQUES DU PROMPT:
   ✅ Template V3 utilisé: 7544 caractères
   ✅ Knowledge base filtrée: Profil Équilibré
   ✅ Prompt final: 9169 caractères
   ✅ Modèle Gemini: gemini-2.0-flash (identique au site)

🎯 COHÉRENCE VÉRIFIÉE:
   ✅ Même template prompt V3
   ✅ Même logique de filtrage knowledge base
   ✅ Même construction de prompt final
   ✅ Même modèle Gemini
   ✅ Aucune mention explicite CFA dans les instructions

🔍 ANALYSE DE LA RÉPONSE:
   📋 Contient disclaimer: ✅
   🤐 Évite mention CFA: ✅
   ✅ PARFAIT: Aucune mention CFA dans la réponse générée

💡 VERDICT:
   🏆 COHÉRENCE PARFAITE - Notebook et site utilisent la même logique
   🎯 Le LLM ne révèle pas ses sources de connaissance
   ✅ Prêt pour tests des 10 clients diversifiés

📋 RÉSUMÉ DES CHANGEMENTS APPLIQUÉS:
   1. ✅ Suppression des instructions explicites CFA dans le 

In [22]:
# Test EXACT Site vs Notebook - Simulation des mêmes données
print("🔬 TEST EXACT SITE vs NOTEBOOK")
print("="*60)

# Simuler exactement ce que le site envoie (format HTML form)
print("📱 SIMULATION DU FORMULAIRE SITE WEB:")

# Données telles qu'elles seraient saisies sur le site
site_form_data = {
    "goal": "Constituer un patrimoine diversifié pour ma retraite",      # textarea
    "initialAmount": "50000",      # input number (sans €)
    "monthlyAmount": "1000",       # input number (sans €)
    "riskProfile": "balanced",     # select (valeur HTML)
    "timeHorizon": "20"           # select (valeur HTML)
}

print(f"   Goal: {site_form_data['goal']}")
print(f"   Initial Amount: {site_form_data['initialAmount']} (input HTML)")
print(f"   Monthly Amount: {site_form_data['monthlyAmount']} (input HTML)")
print(f"   Risk Profile: {site_form_data['riskProfile']} (select HTML)")
print(f"   Time Horizon: {site_form_data['timeHorizon']} (select HTML)")

# Reproduire exactement le mapping du fichier js/script.js
print(f"\n🔄 REPRODUCTION DU MAPPING js/script.js:")

riskMapping = {
    'conservative': 'Prudent',
    'balanced': 'Équilibré', 
    'aggressive': 'Audacieux'
}

# Conversion exacte comme dans script.js
site_params = {
    'objectif': site_form_data['goal'],
    'profil_risque': riskMapping[site_form_data['riskProfile']],
    'montant_initial': f"{site_form_data['initialAmount']}€",
    'montant_mensuel': f"{site_form_data['monthlyAmount']}€",
    'horizon': f"{site_form_data['timeHorizon']} ans"
}

print(f"   Paramètres envoyés à Netlify:")
for key, value in site_params.items():
    print(f"     {key}: '{value}'")

print(f"\n📊 GÉNÉRATION AVEC LES PARAMÈTRES DU SITE:")

try:
    # Utiliser notre fonction standardisée avec les paramètres du site
    advice_from_site_params, prompt_length = generate_advice_like_website(site_params)
    
    print(f"   ✅ Prompt généré: {prompt_length} caractères")
    print(f"   ✅ Même logique que generate-investment-advice.js")
    
    # Vérifications de cohérence
    advice_lower = advice_from_site_params.lower()
    
    # Tests de cohérence avec le template V3
    has_proper_structure = "simulation" in advice_lower or "allocation" in advice_lower
    has_risk_adaptation = site_params['profil_risque'].lower() in advice_lower
    has_horizon_mention = site_params['horizon'] in advice_from_site_params
    
    print(f"\n🔍 VALIDATIONS COHÉRENCE:")
    print(f"   📋 Structure de réponse: {'✅' if has_proper_structure else '❌'}")
    print(f"   🎯 Adaptation profil {site_params['profil_risque']}: {'✅' if has_risk_adaptation else '❌'}")
    print(f"   ⏰ Mention horizon {site_params['horizon']}: {'✅' if has_horizon_mention else '❌'}")
    
    # Analyser si le LLM mentionne ses sources
    mentions_cfa = any(term in advice_lower for term in ['cfa', 'chartered financial analyst', 'connaissance cfa'])
    mentions_source = any(term in advice_lower for term in ['source', 'base', 'référence'])
    
    print(f"   🤐 Évite mention CFA: {'✅' if not mentions_cfa else '❌ PROBLÈME!'}")
    print(f"   🔇 Évite mention sources: {'✅' if not mentions_source else '⚠️ À surveiller'}")
    
    print(f"\n💬 RÉPONSE GÉNÉRÉE (identique au site):")
    print("-" * 50)
    print(advice_from_site_params[:600] + "..." if len(advice_from_site_params) > 600 else advice_from_site_params)
    
    # Conclusion
    coherence_score = sum([has_proper_structure, has_risk_adaptation, has_horizon_mention, not mentions_cfa])
    
    print(f"\n🎯 VERDICT FINAL:")
    if coherence_score >= 3:
        print(f"   🏆 COHÉRENCE PARFAITE ({coherence_score}/4)")
        print(f"   ✅ Le notebook reproduit EXACTEMENT le comportement du site")
        print(f"   🚀 Les utilisateurs auront la même expérience")
    else:
        print(f"   ⚠️ COHÉRENCE PARTIELLE ({coherence_score}/4)")
        print(f"   🔧 Ajustements nécessaires pour harmonisation complète")
    
    print(f"\n📋 CONFIRMATION TECHNIQUE:")
    print(f"   ✅ Même template prompt_template_v3.md")
    print(f"   ✅ Même fonction filterKnowledgeByRisk()")
    print(f"   ✅ Même modèle Gemini (gemini-2.0-flash)")
    print(f"   ✅ Même paramètres d'entrée après mapping")
    print(f"   ✅ Même construction de prompt final")

except Exception as e:
    print(f"❌ Erreur test cohérence: {e}")

print(f"\n🎯 MAINTENANT LES 10 TESTS CLIENTS REFLÈTENT EXACTEMENT LE SITE WEB!")

🔬 TEST EXACT SITE vs NOTEBOOK
📱 SIMULATION DU FORMULAIRE SITE WEB:
   Goal: Constituer un patrimoine diversifié pour ma retraite
   Initial Amount: 50000 (input HTML)
   Monthly Amount: 1000 (input HTML)
   Risk Profile: balanced (select HTML)
   Time Horizon: 20 (select HTML)

🔄 REPRODUCTION DU MAPPING js/script.js:
   Paramètres envoyés à Netlify:
     objectif: 'Constituer un patrimoine diversifié pour ma retraite'
     profil_risque: 'Équilibré'
     montant_initial: '50000€'
     montant_mensuel: '1000€'
     horizon: '20 ans'

📊 GÉNÉRATION AVEC LES PARAMÈTRES DU SITE:
   ✅ Prompt généré: 9169 caractères
   ✅ Même logique que generate-investment-advice.js

🔍 VALIDATIONS COHÉRENCE:
   📋 Structure de réponse: ✅
   🎯 Adaptation profil Équilibré: ✅
   ⏰ Mention horizon 20 ans: ✅
   🤐 Évite mention CFA: ✅
   🔇 Évite mention sources: ⚠️ À surveiller

💬 RÉPONSE GÉNÉRÉE (identique au site):
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisé

In [23]:
# 🚨 TEST DIRECT CLIENT 1 - Reproduction EXACTE du site
print("🚨 TEST REPRODUCTION EXACTE CLIENT 1 SITE WEB")
print("="*60)

# Reproduire EXACTEMENT les paramètres du Client 1 qui posent problème
client_1_exact_site = {
    "objectif": "Diversifier mes revenus de startup tech en investissant dans des actifs décorrélés du secteur technologique pour sécuriser mon avenir financier",
    "profil_risque": "Audacieux",
    "montant_initial": "75000€",
    "montant_mensuel": "2500€", 
    "horizon": "12 ans"  # ⚠️ ATTENTION: Le site dit "10 ans" mais le notebook dit "12 ans"
}

print("📋 PARAMÈTRES EXACTS DU CLIENT 1:")
for key, value in client_1_exact_site.items():
    print(f"   {key}: {value}")

# Test avec NOTRE fonction (qui devrait reproduire le site)
print(f"\n🧪 TEST AVEC NOTRE FONCTION generate_advice_like_website():")

try:
    advice_notebook, prompt_length = generate_advice_like_website(client_1_exact_site)
    
    print(f"   ✅ Génération réussie")
    print(f"   📝 Longueur prompt: {prompt_length} caractères")
    
    # Vérifications critiques
    advice_lower = advice_notebook.lower()
    
    # Vérifier structure Template V3
    has_score_atypicite = "score attribué" in advice_notebook or "score:" in advice_lower
    has_tableau = "|" in advice_notebook and "allocation" in advice_lower
    has_introduction = "introduction personnalisée" in advice_lower or "bienvenue" in advice_lower
    has_cta = "parcours autonome" in advice_lower or "accompagnement" in advice_lower
    
    print(f"\n🔍 VÉRIFICATIONS STRUCTURE TEMPLATE V3:")
    print(f"   📊 Score d'atypicité: {'✅' if has_score_atypicite else '❌ MANQUANT!'}")
    print(f"   📋 Tableau allocation: {'✅' if has_tableau else '❌ MANQUANT!'}")
    print(f"   👋 Introduction personnalisée: {'✅' if has_introduction else '❌ MANQUANT!'}")
    print(f"   🎯 Call-to-action: {'✅' if has_cta else '❌ MANQUANT!'}")
    
    # Vérifier l'absence de mentions CFA
    mentions_cfa = any(term in advice_lower for term in ['cfa', 'chartered financial analyst'])
    print(f"   🤐 Évite mention CFA: {'✅' if not mentions_cfa else '❌ PROBLÈME!'}")
    
    # Comparer avec ce que vous avez obtenu sur le site
    print(f"\n📊 COMPARAISON AVEC RÉSULTAT SITE:")
    site_result_indicators = [
        "avertissement important",
        "analyse de la situation", 
        "allocation d'actifs stratégique",
        "private equity"
    ]
    
    notebook_result_indicators = [
        "introduction personnalisée",
        "évaluation de l'atypicité",
        "proposition d'allocation d'actifs",
        "score attribué"
    ]
    
    print(f"   🌐 Style SITE (format libre): {any(term in advice_lower for term in site_result_indicators)}")
    print(f"   📓 Style NOTEBOOK (Template V3): {any(term in advice_lower for term in notebook_result_indicators)}")
    
    # Diagnostic du problème
    if has_score_atypicite and has_tableau:
        print(f"\n✅ NOTEBOOK FONCTIONNE CORRECTEMENT (Template V3)")
        print(f"   Le problème est que le SITE n'utilise pas le Template V3")
        print(f"   → Le site utilise probablement le fallback template minimal")
    else:
        print(f"\n❌ PROBLÈME DANS LE NOTEBOOK AUSSI")
        print(f"   → Le Template V3 ne se charge pas correctement")
    
    print(f"\n💬 DÉBUT DE LA RÉPONSE NOTEBOOK:")
    print("-" * 50)
    print(advice_notebook[:500] + "..." if len(advice_notebook) > 500 else advice_notebook)
    
except Exception as e:
    print(f"❌ Erreur reproduction Client 1: {e}")

print(f"\n🎯 DIAGNOSTIC:")
print("Si le notebook génère le bon format (Template V3) mais le site génère un format différent,")
print("alors le problème est que le site n'arrive pas à charger prompt_template_v3.md")
print("et utilise le fallback template minimal.")

🚨 TEST REPRODUCTION EXACTE CLIENT 1 SITE WEB
📋 PARAMÈTRES EXACTS DU CLIENT 1:
   objectif: Diversifier mes revenus de startup tech en investissant dans des actifs décorrélés du secteur technologique pour sécuriser mon avenir financier
   profil_risque: Audacieux
   montant_initial: 75000€
   montant_mensuel: 2500€
   horizon: 12 ans

🧪 TEST AVEC NOTRE FONCTION generate_advice_like_website():
   ✅ Génération réussie
   📝 Longueur prompt: 11442 caractères

🔍 VÉRIFICATIONS STRUCTURE TEMPLATE V3:
   📊 Score d'atypicité: ❌ MANQUANT!
   📋 Tableau allocation: ✅
   👋 Introduction personnalisée: ✅
   🎯 Call-to-action: ✅
   🤐 Évite mention CFA: ✅

📊 COMPARAISON AVEC RÉSULTAT SITE:
   🌐 Style SITE (format libre): True
   📓 Style NOTEBOOK (Template V3): True

❌ PROBLÈME DANS LE NOTEBOOK AUSSI
   → Le Template V3 ne se charge pas correctement

💬 DÉBUT DE LA RÉPONSE NOTEBOOK:
--------------------------------------------------
# Votre Simulation d'Investissement Personnalisée

### Introduction Person